In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
datasets = [
# "BEARING_Univar",
# "Control_charts",
"PHM2022_Multivar", #Starting to work on PHM2022_Multivar_Dataset at 2023-06-14 09:18:21  

]

datasets_path = "../datasets"

for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy")
    Dataset = Dataset
    print(Dataset.shape)
    
    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy")
    Labels = Labels.squeeze()
    print(Labels.shape)

(53785, 749, 3)
(53785,)


In [3]:
from dl4tsc.classifiers.GASF import GASF_enc

In [4]:
P=min(Dataset.shape[1], 128)
Dataset = GASF_enc(Dataset, P)
Dataset.shape

(53785, 128, 128, 3)

In [ ]:
# transform the labels from integers to one hot vectors
enc = OneHotEncoder(categories='auto')
Labels = enc.fit_transform(Labels.reshape(-1, 1)).toarray()
Labels.shape

In [ ]:
N = Dataset.shape[0]
T = Dataset.shape[1]
M = Dataset.shape[2]
P = 60 #Length of the approximated representation

Dataset_PAA = np.zeros((N,P, M))
from pyts.approximation import PiecewiseAggregateApproximation
# PAA transformation
paa = PiecewiseAggregateApproximation(window_size=None, output_size=P)
for i in range(M):
    X_paa = paa.transform(Dataset[:,:,i])
    Dataset_PAA[:,:,i] = X_paa

In [ ]:
import matplotlib.pyplot as plt
plt.plot(Dataset_PAA[0,:,0])

In [ ]:
plt.plot(Dataset[0,:,0])

In [ ]:
# Dataset = Dataset[:,:,0]
# Dataset.shape

In [ ]:
Dataset_PAA.shape

In [ ]:
np.unique(Labels)

In [ ]:
# Recurrence plot transformation
from pyts.image import GramianAngularField
transformer = GramianAngularField()
Dataset_PAA_GAF = np.zeros((N,P,P,M))

for i in range(M):
    transformer.fit(Dataset_PAA[:,:,i])
    X_paa_gaf = transformer.transform(Dataset_PAA[:,:,i])
    Dataset_PAA_GAF[:,:,:,i] = X_paa_gaf


In [ ]:
Dataset_PAA_GAF.shape

In [ ]:
del Dataset_PAA

In [ ]:
del Dataset

In [ ]:
import gc

# Perform your code and create some objects

# Call the garbage collector
gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Dataset_PAA_GAF, Labels, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
import matplotlib.pyplot as plt

# Define the label you want to add to the plot
label_text = 'Label: ' + str(y_train[6])

# Show the results for the first time series
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(X_train[6,:,:,0], cmap='hsv', origin='lower')
ax.set_title('GAF Plot', fontsize=18)
ax.text(0, 1, label_text, transform=ax.transAxes, fontsize=12, va='top', ha='left')
plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras as keras


input_shape = X_train.shape[1:]
nb_classes = len(np.unique(Labels, axis=0))

input_layer = keras.layers.Input(input_shape)

# Convolutional layers
conv1 = keras.layers.Conv2D(64, (3, 3), activation='relu')(input_layer)
pooling1 = keras.layers.MaxPooling2D((2, 2))(conv1)

conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu')(pooling1)
pooling2 = keras.layers.MaxPooling2D((2, 2))(conv2)

# Flatten the output
flatten = keras.layers.Flatten()(pooling2)

# Fully connected layers
dense1 = keras.layers.Dense(128, activation='relu')(flatten)
dense2 = keras.layers.Dense(64, activation='relu')(dense1)

output_layer = keras.layers.Dense(nb_classes, activation='softmax')(dense2)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=30,batch_size=32 ,validation_data=(X_test, y_test))
